In [ ]:
%matplotlib notebook

import numpy as np
import random
import IPython
import time
import pandas as pd
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt
import cobra
from cobra.flux_analysis import find_blocked_reactions
import imageio       
import pickle
import sys
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from fipy import CellVariable, Grid2D, Viewer, TransientTerm, DiffusionTerm, Variable, ImplicitSourceTerm
from fipy.tools import numerix
import copy
# print(sys.path)



In [ ]:
class BacteriaData:
    default_params = {
        "speed": 10,
        "theta_diffusion": 0.8,
        "c": "sandybrown",
        "anti": 20
    }
    def __init__(self, total_bacteria, x_width, y_width, pillar_h):
        self.pos = np.random.uniform(0, x_width, size=(total_bacteria,2))
        self.pos[:, 1] = np.random.uniform(pillar_h, y_width, size=(total_bacteria,))
        self.speed = np.ones(total_bacteria)
        self.theta = np.random.uniform(0, 6.2832, total_bacteria)
        self.theta_diffusion = np.ones(total_bacteria)
        self.frozen=np.zeros(total_bacteria).astype(bool)
        self.e_type=np.zeros(total_bacteria)
        self.color = np.empty(total_bacteria, dtype=object)
        self.anti_su=np.empty(total_bacteria, dtype=object)
        self.active=np.zeros(total_bacteria, dtype=bool)
        self.mass = np.ones(total_bacteria)
        self.anti=np.zeros(total_bacteria)
     
    def gen_batch(self, e_type, batch_size, x_width, y_width, pillar_h):
        pos = np.zeros((batch_size,2))
        pos[:, 0] = np.random.uniform(0, x_width-1e-3, size=(batch_size,))
        pos[:, 1] = np.random.uniform(pillar_h, y_width, size=(batch_size,))
        s = np.zeros(batch_size) + self.default_params["speed"]
        theta = np.random.uniform(0, 6.2832, batch_size)
        theta_diffusion = np.zeros(batch_size) + self.default_params["theta_diffusion"]
        frozen = np.zeros(batch_size, dtype=bool)
        e_type = np.zeros(batch_size) + e_type
        color = np.array([self.default_params["c"]*batch_size])
        mass = np.ones(batch_size)
        anti=np.zeros(batch_size) + self.default_params["anti"]
        return (pos, s, theta, theta_diffusion, frozen, e_type, color, mass, anti)
            
    def spawn_batch(self, pos, speed, theta, theta_diffusion, frozen, e_type, color, mass, anti):
        open_ind, = np.nonzero(~self.active)
        batch_size = pos.shape[0]
        batch_ind = open_ind[:batch_size] 
        self.pos[batch_ind, :] = pos
        self.speed[batch_ind] = speed
        self.theta[batch_ind] = theta
        self.theta_diffusion[batch_ind] = theta_diffusion
        self.frozen[batch_ind] = frozen
        self.e_type[batch_ind] = e_type
        self.color[batch_ind] = color
        self.mass[batch_ind] = mass
        self.active[batch_ind] = True
        self.anti[batch_ind]=anti
        

In [ ]:
def render_bd(ax, fig, bd, structure_grid, gx, gy, output_conc_grid, x_width, y_width, saving=False):
    ax.clear()
    ax.set_xlim(0,x_width)
    ax.set_ylim(0,y_width)
    pos = bd.pos[bd.active]
    color = bd.color[bd.active]
    b_rad = np.sqrt(bd.mass[bd.active])
    s = (4*72/500*b_rad)**2
    ax.pcolormesh(gx, gy, (output_conc_grid),  shading='flat', vmin=0, vmax=0.0001, cmap=plt.get_cmap("Blues")) #Render structure grid
    ax.pcolormesh(gx, gy, structure_grid.T, shading='flat', alpha=0.3, vmin=0, vmax=2, cmap=plt.get_cmap("binary"))
    ax.scatter(pos[:,0], pos[:,1], marker='o', c=color, s=s)
    
    # clear_output(wait=True)
    fig.canvas.draw()
    if saving: fig.savefig('plot.png', dpi=300)

In [ ]:
def shoving_update(pos, b_rad):
    if(pos.shape[0] == 0):
        return pos
    ft = KDTree(pos, leaf_size=2)
    neighbor_inds, dist = ft.query_radius(pos, b_rad.max()+b_rad, return_distance=True) #np.array([neighbors_i])
    overlap_delta = np.zeros(pos.shape)
    for ind, ns, dist, rad in zip(range(neighbor_inds.shape[0]), neighbor_inds, dist, b_rad):
        overlap = dist - b_rad[ns] - rad
        q = (overlap < 0) & (ns != ind)
        if(q.sum()):
            ns = ns[q]
            neighbor_delta = (pos[ns, :] - pos[ind, :])   
            neighbor_delta = neighbor_delta / (1e-7 + dist[q].reshape((-1, 1)))*overlap[q].reshape((-1, 1))
            td = neighbor_delta.sum(axis=0)
            overlap_delta[ind, :] = td
    return overlap_delta/2.0

In [ ]:
def shoving_update_numba(pos, b_rad, neighbor_inds, dist):
    if(pos.shape[0] == 0):
        return pos
    N, k = neighbor_inds.shape 
    overlap_delta = np.zeros(pos.shape)
    for i in range(N):
        count = 0
        for j in range(k):
            ni = neighbor_inds[i, j]
            overlap_dist = dist[i, j] - b_rad[i] - b_rad[ni]
            if(dist[i, j] != 0 and overlap_dist < 0):
                neighbor_delta = pos[ni, :] - pos[i, :]

                neighbor_delta = (neighbor_delta*overlap_dist) / dist[i, j]
                overlap_delta[i, :] += neighbor_delta
                count += 1
        if(count > 0):
            overlap_delta[i, :] /= count
    return overlap_delta

def f_shove(pos, b_rad):
    ft = KDTree(pos, leaf_size=2)
    dist, neighbor_inds = ft.query(pos, k=min(9, pos.shape[0]))
    return shoving_update_numba(pos, b_rad, neighbor_inds, dist)/2.0


In [ ]:
#collison with solid objects
def update_with_collisions(pos, next_pos, grid_size, structure_grid):
    #Get grid square of updated position
    next_cell_num = np.mod((next_pos // grid_size).astype(int), structure_grid.shape)

    #Freeze bacteria which would have entered frozen cells. 
    now_frozen = structure_grid[next_cell_num[:, 0], next_cell_num[:, 1]].astype(bool)
    
    #   #Calculate time of collision with each side of grid cell, and take minimum positive time
    f_pos = pos[now_frozen]
    f_next_pos = next_pos[now_frozen]
    delta = f_next_pos - f_pos + 1e-10
    isct_pos_ll = next_cell_num[now_frozen] * grid_size
    t_isct_ll = (isct_pos_ll - f_pos) / delta
    t_isct_ur = (grid_size + isct_pos_ll - f_pos) / delta
    t_isct = np.concatenate([t_isct_ll, t_isct_ur], axis=1) 
    t_isct[t_isct < 0] = 1
    t_isct = np.min(t_isct, axis=1).reshape((-1, 1))
    
    updated_pos = next_pos.copy()
    updated_pos[now_frozen] = pos[now_frozen] + delta*t_isct
    return updated_pos, now_frozen

#Calculates the grid density for a given bacterial population
def get_cell_density(pos, mass, grid_size, x_width, y_width):
    
    edge = (y_width // grid_size) - 1
    pos_nm = np.floor(pos / grid_size)
    pos_nm = np.clip(pos_nm, 0, edge).astype(np.int64)
    
    cell_density=np.zeros((int(y_width // grid_size), int(x_width // grid_size))) 
    for i in range(len(pos_nm[:, 0])):      
        cell_density[pos_nm[i,1], pos_nm[i,0]] += mass[i]

    return cell_density



In [ ]:
class BacteriaField:
    
    def __init__(self, grid_size, x_width, y_width, initial_bulk_conc, pillar_dia, pillar_spacing, pillar_h,
                 bottom_margin, top_faces, bot_faces, diff_c, bf_Df, mw_values, iteration_interval, structure_grid):
        
        self.x_width = x_width
        self.y_width = y_width
        self.grid_size = grid_size
        self.nx = x_width // grid_size
        self.ny = y_width // grid_size
        self.mesh = Grid2D(dx=grid_size, dy=grid_size, nx=self.nx, ny=self.ny)
        self.top_faces = top_faces
        self.bot_faces = bot_faces
        self.dt = iteration_interval
        self.structure_grid = structure_grid.transpose().flatten()
        #diffusion coefs
        self.diff_c      = diff_c
        self.bf_Df       = bf_Df

        self.diffusion_coef = []
        for met in range(len(self.diff_c)):
            Dcoef = CellVariable(name = f"diffusion_C{met}", mesh = self.mesh, value=0)
            self.diffusion_coef.append(Dcoef) 
            
        self.concentrations = []
        for met in range(len(initial_bulk_conc)):
            concentration_var = CellVariable(name=f"concentration{met}", mesh=self.mesh)
            concentration_var.setValue(initial_bulk_conc[met], where=self.structure_grid.T.flatten()==0) 
            if top_faces[met] == 'out':
                concentration_var.faceGrad.constrain(0, where=self.mesh.facesTop)
            else:
                concentration_var.constrain(top_faces[met], self.mesh.facesTop)
            if bot_faces[met] != 'None':
                concentration_var.constrain(bot_faces[met], self.mesh.facesBottom)
            self.concentrations.append(concentration_var)
        
        self.consumption_rate = CellVariable(name = "consumption_rate", mesh = self.mesh, value = 0.0)

    def diffusion_step(self, time, batch_t, consumption_grid, agent_pos, grid_size, mw_values,
                       mets, microbes, vmax, km, opt_method): 
        opt_fluxes = np.zeros((len(mets), len(microbes), self.nx*self.ny))
        mo_mat = np.zeros((len(microbes), self.nx*self.ny))
        flux_distr = [[[] for _ in range(self.nx * self.ny)] for _ in range(len(microbes))] 

        conc_in_mmol = np.array(self.concentrations) * 1e18 #/ mw_values.reshape(-1, 1) #* 4e6 #bac/um3 to mmol/m3
        conc_in_mmol[conc_in_mmol < 0] = 0
        
        consumption_frozen = []         
        flatten_grid_met = [grid.flatten() for grid in consumption_grid] 
        max_metReq = np.array([[v * grid for v in vmax] for grid in flatten_grid_met]) * 60 * self.dt
        all_metReq = np.sum(max_metReq, axis=0)
        compare_conc = (all_metReq > conc_in_mmol).astype(int)
        input_conc = np.where(compare_conc == 1, conc_in_mmol / len(microbes), conc_in_mmol)

        for ind_b, sbml_model in enumerate(microbes):
            grid_loc = consumption_grid[ind_b].flatten()
            agent_pos_i = (np.floor(agent_pos[ind_b] / grid_size)).astype(np.int64)
            pos_flatten = agent_pos_i[:, 0]  + agent_pos_i[:, 1] * self.ny
            agents_pos = np.where(grid_loc!= 0, 1, 0)
            consumption_frozen.append(np.zeros(len(agent_pos[ind_b])))

            print(f"Bacteria {ind_b}: {len(agent_pos[ind_b])} agents active in {len(grid_loc[grid_loc != 0])} blocks")
            for n in range(self.nx*self.ny):
                if grid_loc[n] != 0: 
                    met_concs = [input_conc[met][n] for met in range(len(mets))] 
                    mo, ysx, EX_fluxes = GEM_function1(sbml_model, mets, met_concs, opt_method, vmax, km)
                    flux_distr[ind_b][n] = EX_fluxes
                    opt_fluxes[:, ind_b, n] = EX_fluxes * grid_loc[n] * 1e-18
                    mo_mat[ind_b, n] = mo
                    if time > batch_t:
                        pos_x, pos_y = [n//self.nx, n%self.ny]
                        # convert 1/hr to 1/min
                        consumption_frozen[ind_b][np.where(pos_flatten == n)] = mo / 60
                        
        rxnTerm = np.sum(opt_fluxes, axis=1) / 60
        bf_grid = np.any(np.array(consumption_grid), axis=0).flatten().astype(int)
        initial_conc = self.concentrations
        for eq in range(len(mets)):
            self.consumption_rate.setValue(rxnTerm[eq])
            assert self.consumption_rate.shape == self.concentrations[eq].shape, f"Expected to have shape {self.consumption_rate.shape} but got {concentrations[eq].shape}"
            assert self.diffusion_coef[eq].shape == self.concentrations[eq].shape, f"Expected to have shape {self.diffusion_coef.shape} but got {concentrations[eq].shape}"
            if self.top_faces[eq] == 'out':
                self.concentrations[eq].faceGrad.constrain(0., where=self.mesh.facesTop)
            else:
                self.concentrations[eq].constrain(self.top_faces[eq], self.mesh.facesTop)
            self.diffusion_coef[eq].setValue(self.diff_c[eq] * (1-bf_grid) + self.bf_Df[eq] * bf_grid)
            solver_eq = DiffusionTerm(coeff=self.diffusion_coef[eq]) + self.consumption_rate
            solver_eq.solve(var=self.concentrations[eq])
            if self.bot_faces[eq] != 'None':
                self.concentrations[eq].setValue(self.bot_faces[eq], where=self.structure_grid.T.flatten())
                
        grid_concs = [np.zeros(self.y_width//self.grid_size*self.x_width//self.grid_size) for m_count in range(len(mets))]
        for m_count in range(len(mets)):
            grid_concs[m_count]=self.concentrations[m_count].value.reshape(self.y_width//self.grid_size, self.x_width//self.grid_size)        
            
        return  grid_concs, consumption_frozen, flux_distr

In [ ]:
def simulation(pillar_spacing, pillar_h, grid_size, pillar_dia, x_width, y_width, bottom_margin,
               mets, met_data, microbes, batch_sizes, total_bacteria, 
               pillar_height, opt_method, time_steps, iteration_interval, batch_t, render):

    mw_values   = np.array([met_data[met]['Mw'] for met in mets if met in met_data])
    km          = np.array([met_data[met]['km'] for met in mets if met in met_data])
    vmax        = np.array([met_data[met]['vmax'] for met in mets if met in met_data])
    diff_c      = np.array([met_data[met]['coef_diff'] for met in mets if met in met_data]) 
    bf_Df       = np.array([met_data[met]['bf_Df'] for met in mets if met in met_data])
    
    #BCs and ICs
    initial_bulk_conc = np.array([met_data[met]['initial_con'] for met in mets if met in met_data])
    top_faces = np.array([met_data[met]['top_face'] for met in mets if met in met_data])
    bot_faces = np.array([met_data[met]['bottom_face'] for met in mets if met in met_data])

    x_width, y_width= x_width, y_width 

    bd = BacteriaData(total_bacteria, x_width, y_width, pillar_h)

    #grid-setup  
    structure_grid = np.zeros((x_width // grid_size, y_width // grid_size))
    structure_grid[:, 0:bottom_margin // grid_size]= 1
       
    #sticky surface
    num_pill=int(x_width//(pillar_dia+pillar_spacing))
    for nu in range(1, num_pill+1, 1):
        start = nu * (pillar_spacing // grid_size) + (nu - 1) * (pillar_dia // grid_size)
        end = start + pillar_dia // grid_size
        structure_grid[start:end, 0:pillar_height // grid_size] = 1    
    gx, gy = np.linspace(0, x_width, structure_grid.shape[0]+1), np.linspace(0, y_width, structure_grid.shape[1]+1)

    if render:
        fig = plt.figure(figsize=[4, 4])
        ax = fig.add_subplot(111)
        ax.set_xlim(0,x_width)
        ax.set_ylim(0,y_width)
        plt.ion()
        fig.show()
        fig.canvas.draw()
    
    #initial batch
    Field=BacteriaField(grid_size, x_width, y_width, initial_bulk_conc, pillar_dia, pillar_spacing, pillar_h,
                        bottom_margin, top_faces, bot_faces, diff_c, bf_Df, mw_values, iteration_interval, structure_grid)
    
    bd.active = bd.active & bd.frozen #Deactivate non-frozen bacteria
   
    for i_s in range(len(batch_sizes)):
        bd.spawn_batch(*bd.gen_batch(i_s, batch_sizes[i_s], x_width, y_width, pillar_h))
        bd.spawn_batch(*bd.gen_batch(i_s, int(batch_sizes[i_s]), x_width, y_width, pillar_h))
        bd.spawn_batch(*bd.gen_batch(i_s, int(batch_sizes[i_s]), x_width, y_width, pillar_h))
        bd.spawn_batch(*bd.gen_batch(i_s, int(batch_sizes[i_s]), x_width, y_width, pillar_h))
        bd.spawn_batch(*bd.gen_batch(i_s, int(batch_sizes[i_s]), x_width, y_width, pillar_h))

    t=0-batch_t/60 
    
    totalCon =[]      
    metaboliteConc = []
    agentsInfo = []
    xxx = np.zeros((len(microbes) + 1, time_steps + 1))
    
    consumption_fluxes = []
    for i in range(time_steps):  #i is the iteration number, t shows the time
        print(f"time step: {i} of {time_steps}")

        if((i == batch_t) & (i > 0)):
            bd.active = bd.active & bd.frozen

        #Add diffusion to theta
        theta, theta_diffusion = bd.theta[bd.active], bd.theta_diffusion[bd.active]
        theta += np.random.normal(0, 1, theta.shape)*theta_diffusion

        #Calculate next position
        pos, speed, frozen = bd.pos[bd.active], bd.speed[bd.active], bd.frozen[bd.active]
        delta = np.stack([np.cos(theta), np.sin(theta)], axis=1)*speed.reshape((-1, 1))*(~frozen).reshape((-1, 1))
        next_pos = pos + delta    
        next_pos[:,1] = np.where(next_pos[:,1]>y_width-10, (2*(y_width-10))-next_pos[:,1], next_pos[:,1])
        next_pos, now_frozen = update_with_collisions(pos, next_pos, grid_size, structure_grid)

        frozen = frozen | now_frozen

        #Wrap bacteria around edges
        pos=np.mod(next_pos, np.array([x_width, y_width]))

        #Shove frozen bacteria away from each other
        mass = bd.mass[bd.active]
        b_rad = np.sqrt(mass)*2.1 #Convert from bacteria mass to bacteria radius
        f_pos, f_rad = pos[frozen], b_rad[frozen]
        

        for _ in range(30):
            f_next_pos = f_pos + shoving_update(f_pos, b_rad[frozen])
            f_next_pos, _ = update_with_collisions(f_pos, f_next_pos, grid_size, structure_grid)
            f_pos = f_next_pos
        pos[frozen] = f_pos
     
        #Freeze bacteria close enough to other frozen bacteria
        if(frozen.sum()):
            frozen_tree = KDTree(pos[frozen, :], leaf_size=2)              
            counts = frozen_tree.query_radius(pos, 5, count_only=True)
            frozen = frozen | counts >= 1 #Freeze neighbors of frozen bacteria

        color, e_type, anti= bd.color[bd.active], bd.e_type[bd.active], bd.anti[bd.active]

        colors_hex = generate_color(microbes)
        for n in range(len(colors_hex)):
            color[(e_type==n) & frozen ] = colors_hex[n]
            color[(e_type==n) & ~frozen] = colors_hex[n]
            
 
        consumption_grid=[]
        
        if (i == 0) or (i > batch_t): 
            agent_pos = []
            for bac_t in range(len(microbes)):
                agent_pos.append(pos[frozen & (e_type==bac_t)])
                consumption_grid.append(get_cell_density(agent_pos[bac_t], mass[frozen & (e_type==bac_t)], grid_size, x_width, y_width))
            grid_concs, consumption_frozen, flux_distr = Field.diffusion_step(i, batch_t, consumption_grid, agent_pos,
                            grid_size, mw_values, mets, microbes, vmax, km, opt_method)
            totalCon.append([np.mean(grid_conc) for grid_conc in grid_concs]) 
        
        if i > batch_t: 
            for ind_b in range(len(microbes)):
                mass[e_type==ind_b] += consumption_frozen[ind_b] * mass[e_type==ind_b] * iteration_interval
                xxx[ind_b, i] = len(mass[e_type==ind_b])
        mass = mass*(1 + np.random.normal(0.01, 0.01, mass.shape)*(~frozen)) #Free bacteria replicate fast
        s = mass > 2
        mass[s] = mass[s]/2
        
        #generate batch of newly split bacteria
        rt = np.random.uniform(0, 6.28, s.sum())
        p = pos[s]
        next_p = p + np.random.normal(0, 0.1, p.shape)
        p, _ = update_with_collisions(p, next_p, grid_size, structure_grid) #Prevent bacteria from spawning inside walls
        rep_batch = (p, speed[s], rt, theta_diffusion[s], frozen[s], e_type[s], color[s], mass[s], anti[s])

        #convert numpy back to the dataframe
        bd.pos[bd.active] = pos
        bd.theta[bd.active] = theta
        bd.frozen[bd.active] = frozen
        bd.mass[bd.active] = mass
        bd.color[bd.active] = color

        bd.active[bd.mass <= 0.1] = False  

        bd.spawn_batch(*rep_batch)
        if i>80:
            bd.active[bd.pos[:, 1]>100] = False
        metaboliteConc.append(copy.deepcopy(grid_concs))
        agentsInfo.append(copy.deepcopy(bd))
        consumption_fluxes.append(copy.deepcopy(flux_distr))
        
        #Rendering
        if render:
            render_bd(ax, fig, bd, structure_grid, gx, gy, grid_concs[0],
                          x_width, y_width, saving=False)        
    #save results
    data_to_save = {
    "met_concentrations": metaboliteConc,
    "agentsInfo": agentsInfo,
    "structure_grid": structure_grid,
    "gx": gx,
    "gy": gy,
    "x_width": x_width,
    "y_width": y_width,
    "time_steps": time_steps,
    "fluxes": consumption_fluxes
                    }
    with open('data.pickle', 'wb') as file:
        pickle.dump(data_to_save, file)
    
    
    return structure_grid, totalCon


In [ ]:
def generate_color(agents, darken_factor=0.99, skip_step=3): 
    # darken_factor: makes colors a bit darker (0-1)
    # skip_step: to make more distinct colors
    
    cmap = plt.get_cmap('tab20', len(agents) * skip_step)
    colors = [cmap(i*skip_step) for i in range(len(agents))]  
    colors = [(color[0] * darken_factor, color[1] * darken_factor, color[2] * darken_factor, color[3]) for color in colors]
    colors_hex = [mcolors.rgb2hex(color) for color in colors]

    return colors_hex

In [ ]:
def calculate_lower_flux(concentrations, vmax, km):
    
    input_fluxes = np.zeros(len(concentrations))
    numerator = vmax * concentrations
    denumerator = km + concentrations
    input_fluxes = numerator/denumerator
    for fl, flux in enumerate(input_fluxes):
        if flux == 0:
            input_fluxes[fl] = concentrations[fl]
        
    return input_fluxes

def apply_lower_flux(model, fluxes, all_reaction_indices):

    for i in range(len(all_reaction_indices)):
        model.reactions.get_by_id(all_reaction_indices[i]).lower_bound = -fluxes[i]

    return model  

def perform_opt(model, opt_method):
    
    if opt_method == 'fba':
        solution = model.optimize() #FBA optimization
    elif opt_method =='pfba':
        solution = cobra.flux_analysis.pfba(model)
#     if solution is not None:
#         print("Optimal solution:", solution.objective_value)
#     else:
#         print("No solution found.")
    return solution

def get_optimized_fluxes(all_reaction_indices, model, solution): 
    total_flux = solution.fluxes[all_reaction_indices] 
    Biomass_flux = solution.fluxes['Growth']#['EX_biomass(e)']
    return np.array(total_flux), np.array(Biomass_flux)

    
def get_flux_parameters(metabolite, met_data):
    vmax = []
    km = []
    for met in metabolite:
        if met in met_data:
            if 'vmax' in met_data[met] and 'km' in met_data[met]:
                vmax.append(met_data[met]['vmax'])
                km.append(met_data[met]['km'])
            else:
                print(f"Warning: Missing data for metabolite: {met}")
        else:
            print(f"Warning: Metabolite {met} not found in metabolites database")

    return np.array(vmax), np.array(km)


In [ ]:
def GEM_function1(model, List_of_metabolites, concs, opt_method, vmax, km):  #we have to give this list of entire metabolites
    
    exchange_fluxes = calculate_lower_flux(concs, vmax, km) #list of concs, vmax, km
    all_reaction_indices = ["EX_"+ met for met in List_of_metabolites] 
    model = apply_lower_flux(model, exchange_fluxes, all_reaction_indices)
    
    # Perform FBA calculation
    solution = perform_opt(model, opt_method)
    opt_exchange_fluxes, biomass_flux = get_optimized_fluxes(all_reaction_indices, model, solution) # flux of the metabolites & biomass producing reaction

    if biomass_flux == 0:
        Ysx = 0
    else:
        Ysx = opt_exchange_fluxes/biomass_flux
           
    return np.array(biomass_flux), np.array(Ysx), np.array(opt_exchange_fluxes)

In [ ]:
model1 = cobra.io.read_sbml_model("path_to_your_model1.xml")
model2 = cobra.io.read_sbml_model("path_to_your_model2.xml")
model3 = cobra.io.read_sbml_model("path_to_your_model3.xml")
model4 = cobra.io.read_sbml_model("path_to_your_model4.xml")
#add all SBML models for the simulations here

In [ ]:
#make a list of GEM models generated

Models = # for example: [model1, model2, ...]

In [ ]:
#identify orphan metabolites manually
def find_orphans(model):
    connected_metabolites = set()
    for reaction in model.reactions:
        connected_metabolites.update(reaction.metabolites.keys())
    return [met for met in model.metabolites if met not in connected_metabolites]

#validate model
def validate_model(model):
    results = {}

    #check blocked reactions
    blocked_reactions = find_blocked_reactions(model)
    results["blocked_reactions"] = len(blocked_reactions)

    #check orphan metabolites
    orphan_metabolites = find_orphans(model)
    results["orphan_metabolites"] = len(orphan_metabolites)

    #check objective function optimization
    try:
        solution = model.optimize()
        results["objective_value"] = solution.objective_value
        results["is_feasible"] = solution.objective_value > 0
    except Exception as e:
        results["objective_value"] = None
        results["is_feasible"] = False
        results["error"] = str(e)

    return results

#check all models in memory
for i, model in enumerate(models):
    print(f"Validating model {i + 1}...")
    try:
        #perform validation
        validation_results = validate_model(model)
        
        #print results
        for key, value in validation_results.items():
            print(f"  {key}: {value}")
    except Exception as e:
        print(f"Error validating model {i + 1}: {str(e)}")


In [ ]:
# Main
# RUN the model
def run():



    
    #Geometery
    pillar_spacing = 50 # spaces between pillars
    pillar_height  = 10 # Height
    grid_size  = 10     # lattice cell size
    pillar_dia = 20     
    
    #Domain size
    x_width = 400
    y_width = 120
    bottom_margin = 20
    
    # import metabolites properties: [conc: bac/um3], [D: um2/min], [km: mmol/um3], [vmax: mmol/g/h]
    df = pd.read_excel('metlist.xlsx')
    met_data = df.set_index('metabolite').T.to_dict('list')
    keys = ['km', 'vmax', 'initial_con', 'coef_diff', 'bf_Df', 'Mw', 'top_face', 'bottom_face']
    met_data = {met: dict(zip(keys,values)) for met, values in met_data.items()}
    
    
    mets = ['glc__D_e','ac_e'] # list of metabolites for diffusion equations

    #microbial features [list of all xml models representing species ]
    microbes = models 
    
    
    batch_sizes = [100 for mic in microbes]
    
    opt_method = 'fba' # optimization method (either 'fba' or 'pfba' can be set)
    render = False
    
    total_microbes = 20000
    no_time_steps = 500
    iteration_interval = 5 #min
    batch_t = 10 
    
    conSgrid, totalCon = simulation(pillar_spacing, pillar_height, grid_size, pillar_dia, x_width, y_width, bottom_margin,
               mets, met_data, microbes, batch_sizes, total_microbes, pillar_height,
               opt_method, no_time_steps, iteration_interval, batch_t, render)
    
    df = pd.DataFrame(np.array(totalCon))
    df.to_excel('output_conc.xlsx', index=False)
     
if __name__ == "__main__":
    run()
    